In [1]:
import os

In [2]:
%pwd

'd:\\Sentence_Generation_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Sentence_Generation_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    
    

In [6]:
# Instantiate the configuration
config = ModelTrainerConfig(
     root_dir=Path('artifacts/model_trainer'),
     data_path=Path('artifacts/data_transformation/cleaned_concepts.csv'),
     model_ckpt='meta-llama/Llama-2-7b-chat-hf',  # Correct model ID
    
)

In [7]:
from textgeneration.constants import *
from textgeneration.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


     
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
              # Make sure this is an object and not a string

            
        )

        return model_trainer_config    

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import pandas as pd

try:
    # Initialize the ConfigurationManager
    config_manager = ConfigurationManager()
    
    # Get the model trainer configuration
    model_trainer_config = config_manager.get_model_trainer_config()



    # Load the dataset from the configuration
    dataset = pd.read_csv(model_trainer_config.data_path)

# Check CUDA availability and set device
    device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# Load the model with CPU offloading and 8-bit precision
    model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",  # Change to local path if necessary
    device_map={"": device},  # Manually set device allocation
    load_in_8bit=True,  # Enable 8-bit loading for reduced memory usage
    torch_dtype=torch.float16,# Use mixed precision for memory efficiency
    low_cpu_mem_usage=True,  
    )

# Optionally, compile the model for faster execution (requires PyTorch 2.0+)
    if torch.__version__ >= "2.0":
       model = torch.compile(model)

# Example: Generate text for the first data point
    concept_set = dataset['cleaned_concept_set'][3]  # Example concept set  
# Remove tags from concepts (keeping only the base word)
#cleaned_concepts = [concept.split('_')[0] for concept in concept_set]

# Create prompt
    prompt = f"Generate a meaningful paragraph with at least 750 words using the following concepts: {''.join(concept_set)}."

# Tokenize the prompt

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)  # Move input_ids to the device

# Set pad_token to eos_token to avoid errors
    tokenizer.pad_token = tokenizer.eos_token

# Generate text
    #with torch.no_grad():  # Disable gradient calculation to save memory

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=config_manager.params['TrainingArguments']['max_length'],
            temperature=config_manager.params['TrainingArguments']['temperature'],
            top_p=config_manager.params['TrainingArguments']['top_p'],
            top_k=config_manager.params['TrainingArguments']['top_k'],
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

# Decode and print the generated text
    generated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Generated Sentence: {generated_sentence}")

except Exception as e:
    # Handle any exceptions and print the error message
    print(f"An error occurred: {e}")
    raise e

# Save the model and tokenizer
model_save_path = os.path.join(model_trainer_config.root_dir, "saved_model")
os.makedirs(model_save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")
 

c:\Users\chopd\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-18 11:11:05,430: INFO: config: PyTorch version 2.4.1+cu118 available.]
[2024-10-18 11:11:05,696: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-18 11:11:05,703: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-18 11:11:05,704: INFO: common: created directory at: artifacts]
[2024-10-18 11:11:05,706: INFO: common: created directory at: artifacts/model_trainer]


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:54<00:00, 27.21s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\chopd\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, wher

Generated Sentence: Generate a meaningful paragraph with at least 750 words using the following concepts: car, drive, road, water.

As I drove along the winding road, I couldn't help but feel a sense of awe at the sheer vastness of the water below. The car, though a trusty companion, seemed small and insignificant in comparison to the endless expanse of the ocean. The road, too, seemed to fade into the distance as I gazed out at the blue horizon. I had always loved driving, but never had I felt such a profound connection to the natural world as I did at that moment. The smooth, paved road seemed to stretch on forever, a testament to the ingenuity of human engineering, yet also a reminder of the fragility of our existence. As I continued to drive, the water stretching out before me grew more turbulent, the waves crashing against the shore with a ferocity that seemed almost primal. I couldn't help but feel a sense of reverence for the power of nature, and a deep appreciation for the humb